In [1]:
from Bio import Seq, SeqIO, SeqRecord
from IPython.display import display
from plotnine import *
from scipy.optimize import curve_fit
from tools import *
import math
import os
import pandas as pd
import random

In [2]:
input_dir = "input"
output_dir = "output"

In [3]:
# Assembly indexing
print(">", "Assembly indexing")
asm_i_dir = input_dir + "/asm"
asm_o_dir = output_dir + "/asm"
if not os.path.exists(asm_o_dir):
    os.makedirs(asm_o_dir)
asms = []
skipped = 0
fls = sorted([i for i in os.listdir(asm_i_dir) if i.endswith(".fa") or fl.endswith(".fasta")])
for fl in fls:
    asm = os.path.splitext(fl)[0]
    if asm in asms:
        print("Assembly name duplicated: {}.".format(asm), file=sys.stderr)
        break
    else:
        asms.append(asm)
    #print(">", asm)
    asm_file = asm_o_dir + "/" + asm + ".fa"
    if not os.path.exists(asm_file):
        os.symlink(os.path.abspath(asm_i_dir + "/" + fl), asm_file)
        os.system("samtools faidx {}".format(asm_file))
    else:
        skipped += 1
        #print("{} exists, skipped.".format(asm_file))
if skipped > 0:
    print("{} / {} runs were skipped.".format(skipped, len(fls)))
print("Assembly number: {}".format(len([i for i in os.listdir(asm_o_dir) if i.endswith(".fa")])))

> Assembly indexing
14 / 14 runs were skipped.
Assembly number: 14


In [4]:
# CRISPR identification
print(">", "CRISPR identification")
cri_dir = output_dir + "/crispr"
if not os.path.exists(cri_dir):
    os.makedirs(cri_dir)
skipped = 0
fls = sorted([i for i in os.listdir(asm_o_dir) if i.endswith(".fa")])
for fl in fls:
    asm = os.path.splitext(fl)[0]
    #print(">", asm)
    asm_file = os.path.join(asm_o_dir, fl)
    # PilerCR
    if os.path.exists("{}/{}_pilerCR.out".format(cri_dir, asm)):
        skipped += 1
        #print("{}/{}_pilerCR.out exists, skipped.".format(cri_dir, asm))
    else:
        pcmd = "pilercr -minid 0.85 -mincons 0.8 -noinfo -in {} -out {}/{}_pilerCR.out 2>{}/{}_pilerCR.err".format(asm_file, cri_dir, asm, cri_dir, asm)
        retCode1 = os.system(pcmd)
        if retCode1 != 0:
            print(str(retCode1) + "  " + pcmd)
            break
    # minced
    if os.path.exists("{}/{}_minCED.out".format(cri_dir, asm)):
        skipped += 1
        #print("{}/{}_minCED.out exists, skipped.".format(cri_dir, asm))
    else:
        mcmd = "minced -minRL 16 -maxRL 64 -minSL 8 -maxSL 64 -searchWL 6 {} {}/{}_minCED.out 2>{}/{}_minCED.err".format(asm_file, cri_dir, asm, cri_dir, asm)
        retCode2 = os.system(mcmd)
        if retCode2 != 0:
            print(str(retCode2) + "  " + mcmd)
            break
if skipped > 0:
    print("{} / {} runs were skipped.".format(skipped, len(fls) * 2))

> CRISPR identification
28 / 28 runs were skipped.


In [5]:
# CRISPR res extraction
print(">", "CRISPR res extraction")
all_dir = output_dir + "/all"
if not os.path.exists(all_dir):
    os.makedirs(all_dir)
criRes = pd.DataFrame()
conRes = pd.DataFrame()
for fl in sorted(os.listdir(cri_dir)):
    #asm = os.path.splitext(fl)[0]
    #print(">", asm)
    if "pilerCR.out" in fl:
        crip, copp = readPIL(os.path.join(cri_dir, fl))
        criRes = pd.concat([criRes, crip], ignore_index=True)
        conRes = pd.concat([conRes, copp], ignore_index=True)
    elif "minCED.out" in fl:
        crip, copp = readMIN(os.path.join(cri_dir, fl))
        criRes = pd.concat([criRes, crip], ignore_index=True)
        conRes = pd.concat([conRes, copp], ignore_index=True)
    elif fl.endswith(".err"):
        continue
    else:
        print("Unrecognized CRISPR identification file: {}".format(os.path.join(cri_dir, fl)), file=sys.stderr)
print("criRes", criRes.shape)
validateSeq(criRes, genomeDir=asm_o_dir)
print("conRes", conRes.shape)
print("{} / {} assemblies contain CRISPRs.".format(len(criRes["asm"].unique()), len(asms)))
criNrs = criRes.pivot_table(index=["cid", "asm"], columns="component", values="id", aggfunc=len).reset_index()
criPos = criRes.groupby(["cid", "asm"]).agg({"contig": lambda x: x.unique()[0], "start": min, "end": max}).reset_index()
criPos["length"] = criPos["end"] - criPos["start"]
criMet = pd.merge(criNrs, criPos.drop(columns="asm"), on="cid")[["contig", "start", "end", "cid", "repeat", "spacer", "length", "asm"]]
repRes = criRes[criRes["component"] == "repeat"]
spaRes = criRes[criRes["component"] == "spacer"]
# write txt
repRes.to_csv(all_dir + "/repeat.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
spaRes.to_csv(all_dir + "/spacer.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
conRes.to_csv(all_dir + "/conRep.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
criMet.to_csv(all_dir + "/criMet.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
# write fa
recs = [SeqRecord.SeqRecord(Seq.Seq(row["seq"]), id=row["id"], description="") for i, row in repRes.iterrows()]
SeqIO.write(recs, all_dir + "/repeat.fa", "fasta")
recs = [SeqRecord.SeqRecord(Seq.Seq(row["seq"]), id=row["id"], description="") for i, row in spaRes.iterrows()]
SeqIO.write(recs, all_dir + "/spacer.fa", "fasta")
recs = [SeqRecord.SeqRecord(Seq.Seq(row["seq"]), id=row["id"], description="") for i, row in conRes.iterrows()]
SeqIO.write(recs, all_dir + "/conRep.fa", "fasta")
# write fa by asm
con_dir = output_dir + "/conRep"
if not os.path.exists(con_dir):
    os.makedirs(con_dir)
for asm, df in conRes.groupby("asm"):
    recs = [SeqRecord.SeqRecord(Seq.Seq(row["seq"]), id=row["id"], description="") for i, row in df.iterrows()]
    SeqIO.write(recs, con_dir + "/" + asm + ".fa", "fasta")

> CRISPR res extraction
criRes (3789, 12)
All the sequences have been validated successfully.
conRes (237, 7)
12 / 14 assemblies contain CRISPRs.


In [6]:
# stat
# repeats or spacers for each asm
stat = criRes.pivot_table(index=["asm", "tool"], columns="component", values="seq",
                          aggfunc=lambda x: (len(x), str(min([len(i) for i in x])) + "-" + str(max([len(i) for i in x])))).reset_index()
stat.columns.name = None
stat.to_csv(all_dir + "/crispr_stat.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
# consensus repeats for each asm
stat_con = conRes.pivot_table(index=["asm", "tool"], values="cid", aggfunc=len).reset_index()
stat_con.to_csv(all_dir + "/conRep_stat.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
# CRISPR number ~ con rep number
stat_arr = criRes.pivot_table(index=["asm", "tool"], values="cid", aggfunc=lambda x: len(x.unique())).reset_index()
assert all(stat_con["cid"] == stat_arr["cid"])

In [7]:
# Blast DB construction
print(">", "Blast DB construction")
skipped = 0
fls = sorted([i for i in os.listdir(asm_o_dir) if i.endswith(".fa")])
for fl in fls:
    asm = os.path.splitext(fl)[0]
    #print(">", asm)
    asm_file = os.path.join(asm_o_dir, fl)
    if not os.path.exists(asm_file + ".ndb"):
        os.system("makeblastdb -in {} -parse_seqids -dbtype nucl > {}".format(asm_file, asm_file + ".log"))
    else:
        skipped += 1
        #print("{} exists, skipped.".format(asm_file + ".ndb"))
if skipped > 0:
    print("{} / {} runs were skipped.".format(skipped, len(fls)))

> Blast DB construction
14 / 14 runs were skipped.


In [8]:
# Con repeat reordering
print(">", "Consensus repeat reordering")
conReo_dir = output_dir + "/conReo"
if not os.path.exists(conReo_dir):
    os.makedirs(conReo_dir)
random.seed(1)
for asm, df in conRes.groupby("asm"):
    recs = []
    for _, row in df.iterrows():
        for i in range(100):
            bases = list(row["seq"])
            random.shuffle(bases)
            reo = "".join(bases)
            recs.append(SeqRecord.SeqRecord(Seq.Seq(reo), id=row["id"] + "_" + str(i + 1), description=""))
    SeqIO.write(recs, conReo_dir + "/" + asm + ".fa", "fasta")

> Consensus repeat reordering


In [9]:
# Blast for consensus repeats (reordered)
print(">", "Blast for consensus repeats (reordered)")
blcReo_dir = output_dir + "/blastConReo"
if not os.path.exists(blcReo_dir):
    os.makedirs(blcReo_dir)
skipped = 0
fls = sorted(os.listdir(conReo_dir))
for fl in fls:
    asm = os.path.splitext(fl)[0]
    #print(">", asm)
    asm_file = os.path.join(asm_o_dir, fl)
    xml_file = os.path.join(blcReo_dir, asm + ".xml")
    if not os.path.exists(xml_file):
        os.system("blastn -query {} -db {} -task blastn-short -dust no -outfmt 5 -num_threads 12 -out {}".format(os.path.join(conReo_dir, fl), asm_file, xml_file))
    else:
        skipped += 1
        #print("{} exists, skipped.".format(xml_file))
if skipped > 0:
    print("{} / {} runs were skipped.".format(skipped, len(fls)))

> Blast for consensus repeats (reordered)
12 / 12 runs were skipped.


In [10]:
# Blast res extraction for consensus repeats (reordered)
print(">", "Blast res extraction for consensus repeats (reordered)")
if not os.path.exists(all_dir + "/blastConReo.txt"):
    blcReop = pd.DataFrame()
    for fl in sorted(os.listdir(blcReo_dir)):
        asm = os.path.splitext(fl)[0]
        #print(">", asm)
        blcReop = pd.concat([blcReop, readBLAST(os.path.join(blcReo_dir, fl))], ignore_index=True)
    blcReop["tool"] = "NA"
    blcReop.loc[blcReop["qseqid"].str.contains("_p_"), "tool"] = "pilerCR"
    blcReop.loc[blcReop["qseqid"].str.contains("_m_"), "tool"] = "minCED"
    blcReop.to_csv(all_dir + "/blastConReo.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
else:
    print("{} exists, read it directly.".format(all_dir + "/blastConReo.txt"))
    blcReop = pd.read_csv(all_dir + "/blastConReo.txt", header=0, sep="\t", quoting=csv.QUOTE_NONE)
print(blcReop.shape)
blcReop.head()

> Blast res extraction for consensus repeats (reordered)
output/all/blastConReo.txt exists, read it directly.
(906605, 20)


,asm,qseqid,sseqid,pident,length,mismatch,gap,qstart,qend,sstart,send,evalue,bitscore,qlen,qcov,strand,qseq,sseq,alm,tool
0,GCA_001594005.1,GCA_001594005.1_m_1_c1_1,LTBA01000010.1,100.0,13,0,0,8,20,10686,10674,0.645306,26.2635,30,0.433,-,TGTTATGATAGGA,TGTTATGATAGGA,|||||||||||||,minCED
1,GCA_001594005.1,GCA_001594005.1_m_1_c1_1,LTBA01000010.1,100.0,12,0,0,10,21,52349,52338,2.549850,24.2811,30,0.400,-,TTATGATAGGAG,TTATGATAGGAG,||||||||||||,minCED
2,GCA_001594005.1,GCA_001594005.1_m_1_c1_1,LTBA01000001.1,100.0,13,0,0,10,22,267085,267097,0.645306,26.2635,30,0.433,+,TTATGATAGGAGG,TTATGATAGGAGG,|||||||||||||,minCED
3,GCA_001594005.1,GCA_001594005.1_m_1_c1_1,LTBA01000001.1,100.0,13,0,0,12,24,344404,344392,0.645306,26.2635,30,0.433,-,ATGATAGGAGGAT,ATGATAGGAGGAT,|||||||||||||,minCED
4,GCA_001594005.1,GCA_001594005.1_m_1_c1_1,LTBA01000001.1,100.0,12,0,0,1,12,290961,290972,2.549850,24.2811,30,0.400,+,TAACTTTTGTTA,TAACTTTTGTTA,||||||||||||,minCED


In [11]:
# stat
# quantile by tool
stat_qt = blcReop.groupby("tool")[["pident", "qcov"]].quantile([i / 100 for i in range(0, 101)]).reset_index()
stat_qt = stat_qt.rename(columns={"level_1": "prob"})
stat_qt95 = stat_qt.loc[stat_qt["prob"] == 0.95, ]
#stat_qt95.to_csv(all_dir + "/blastConReo_stat_qt95.txt", header=True, sep="\t", index=False)
gp1 = ggplot(stat_qt, aes(x="pident", y="prob", group="tool", color="tool")) + geom_line() + geom_point(data=stat_qt.loc[stat_qt["prob"] == 0.95, ]) + \
             xlab("Identity (%)") + ylab("CDF") + ggtitle("Background (reorder)") + theme(aspect_ratio=1)
gp2 = ggplot(stat_qt, aes(x="qcov", y="prob", group="tool", color="tool")) + geom_line() + geom_point(data=stat_qt.loc[stat_qt["prob"] == 0.95, ]) + \
             xlab("Coverage in query") + ylab("CDF") + ggtitle("Background (reorder)") + theme(aspect_ratio=1)
# # quantile by tool and query length
# stat_qt95_tl = blcReop.pivot_table(index=["tool", "qlen"], values=["pident", "qcov"], aggfunc=lambda x: x.quantile(0.95)).reset_index()
# gp3 = ggplot(stat_qt95_tl) + geom_line(aes(x="qlen", y="pident", color="tool")) + xlab("Query length") + ylab("Identity") + labs(color="Tool") + \
#       theme(aspect_ratio=1)
# gp4 = ggplot(stat_qt95_tl) + geom_line(aes(x="qlen", y="qcov", color="tool")) + xlab("Query length") + ylab("Coverage in query") + labs(color="Tool") + \
#       theme(aspect_ratio=1)
# quantile by tool and query length (after average by query)
stat_by_q = blcReop.pivot_table(index=["tool", "qseqid", "qlen"], values=["pident", "qcov"], aggfunc=lambda x: x.mean()).reset_index()
stat_qt95_tla = stat_by_q.pivot_table(index=["tool", "qlen"], values=["pident", "qcov"], aggfunc=lambda x: x.quantile(0.95)).reset_index()
gp3 = ggplot(stat_qt95_tla) + geom_line(aes(x="qlen", y="pident", color="tool")) + scale_y_continuous(limits=(90, 100)) + \
      xlab("Query length") + ylab("Identity") + labs(color="Tool") + theme(aspect_ratio=1)
gp4 = ggplot(stat_qt95_tla) + geom_line(aes(x="qlen", y="qcov", color="tool")) + \
      xlab("Query length") + ylab("Coverage in query") + labs(color="Tool") + theme(aspect_ratio=1)
# # comparison
# stat_cmp = pd.merge(stat_qt95_tl, stat_qt95_tla, on="qlen")
# gp_cmp1 = ggplot(stat_cmp) + geom_point(aes(x="pident_x", y="pident_y"), alpha=0.6) + \
#     scale_x_continuous(limits=(90, 100)) + scale_y_continuous(limits=(90, 100)) + theme(aspect_ratio=1)
# gp_cmp2 = ggplot(stat_cmp) + geom_point(aes(x="qcov_x", y="qcov_y"), alpha=0.6) + geom_abline(slope=1, intercept=0, color="grey", linetype="dashed") + \
#     scale_x_continuous(limits=(0, 1)) + scale_y_continuous(limits=(0, 1)) + theme(aspect_ratio=1)  # qt95 in stat_qt95_tla: small
# quantile by query length
stat_qt95_l = stat_by_q.pivot_table(index=["qlen"], values=["pident", "qcov"], aggfunc=lambda x: x.quantile(0.95)).reset_index()
gp5 = ggplot(stat_qt95_l) + geom_line(aes(x="qlen", y="pident"), color="cornflowerblue") + xlab("Query length") + ylab("Identity") + labs(color="Tool") + \
      theme(aspect_ratio=1)
gp6 = ggplot(stat_qt95_l) + geom_line(aes(x="qlen", y="qcov"), color="cornflowerblue") + xlab("Query length") + ylab("Coverage in query") + labs(color="Tool") + \
      theme(aspect_ratio=1)
# constant mapping (pident ~ qlen)
stat_qt95_l["pident_pred"] = stat_qt95_l["pident"].mean()
# regression (qcov ~ qlen)
def func(x, a, b, c):
    return a ** (x + b) + c
popt, pcov = curve_fit(func, stat_qt95_l["qlen"], stat_qt95_l["qcov"])
stat_qt95_l["qcov_pred"] = func(stat_qt95_l["qlen"], *popt)
stat_qt95_l.to_csv(all_dir + "/blastConReo_stat_qt95.txt", header=True, sep="\t", index=False)
gp7 = ggplot(stat_qt95_l) + geom_line(aes(x="qlen", y="qcov"), color="cornflowerblue") + geom_line(aes(x="qlen", y="qcov_pred"), color="black", size=1) + \
      xlab("Query length") + ylab("Coverage in query") + labs(color="Tool") + \
      theme(aspect_ratio=1)
gps = [gp1, gp2, gp3, gp4, gp5, gp6, gp7]
save_as_pdf_pages(gps, all_dir + "/blastConReo_stat.pdf", verbose=False)

In [12]:
# Blast for consensus repeats
print(">", "Blast for consensus repeats")
blc_dir = output_dir + "/blastCon"
if not os.path.exists(blc_dir):
    os.makedirs(blc_dir)
skipped = 0
fls = sorted(os.listdir(con_dir))
for fl in fls:
    asm = os.path.splitext(fl)[0]
    #print(">", asm)
    asm_file = os.path.join(asm_o_dir, fl)
    xml_file = os.path.join(blc_dir, asm + ".xml")
    if not os.path.exists(xml_file):
        os.system("blastn -query {} -db {} -task blastn-short -dust no -outfmt 5 -num_threads 12 -out {}".format(os.path.join(con_dir, fl), asm_file, xml_file))  # old para
    else:
        skipped += 1
        #print("{} exists, skipped.".format(xml_file))
if skipped > 0:
    print("{} / {} runs were skipped.".format(skipped, len(fls)))

> Blast for consensus repeats
12 / 12 runs were skipped.


In [13]:
# Blast res extraction for consensus repeats
print(">", "Blast res extraction for consensus repeats")
blcp = pd.DataFrame()
for fl in sorted(os.listdir(blc_dir)):
    asm = os.path.splitext(fl)[0]
    #print(">", asm)
    blcp = pd.concat([blcp, readBLAST(os.path.join(blc_dir, fl))], ignore_index=True)  
blcp["tool"] = "NA"
blcp.loc[blcp["qseqid"].str.contains("_p_"), "tool"] = "pilerCR"
blcp.loc[blcp["qseqid"].str.contains("_m_"), "tool"] = "minCED"
blcp.to_csv(all_dir + "/blastCon.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
print(blcp.shape)
blcp.head()

> Blast res extraction for consensus repeats
(43940, 20)


,asm,qseqid,sseqid,pident,length,mismatch,gap,qstart,qend,sstart,send,evalue,bitscore,qlen,qcov,strand,qseq,sseq,alm,tool
0,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,100.000,30,0,0,1,30,79689,79718,4.624340e-11,59.9635,30,1.000,+,ATAAATGGTTCAGGGGATATTAATGGTGAT,ATAAATGGTTCAGGGGATATTAATGGTGAT,||||||||||||||||||||||||||||||,minCED
1,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,100.000,27,0,0,1,27,79740,79766,2.852950e-09,54.0164,30,0.900,+,ATAAATGGTTCAGGGGATATTAATGGT,ATAAATGGTTCAGGGGATATTAATGGT,|||||||||||||||||||||||||||,minCED
2,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,96.154,26,1,0,1,26,79791,79816,2.748110e-06,44.1047,30,0.867,+,ATAAATGGTTCAGGGGATATTAATGG,ATAAATGGTTCAGGAGATATTAATGG,|||||||||||||| |||||||||||,minCED
3,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,100.000,20,0,0,1,20,79638,79657,4.290720e-05,40.1400,30,0.667,+,ATAAATGGTTCAGGGGATAT,ATAAATGGTTCAGGGGATAT,||||||||||||||||||||,minCED
4,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,100.000,12,0,0,3,14,38212,38223,2.549850e+00,24.2811,30,0.400,+,AAATGGTTCAGG,AAATGGTTCAGG,||||||||||||,minCED


In [14]:
# stat (query length ~ hit number)
stat = blcp.pivot_table(index=["asm", "qseqid", "qlen"], values="sseqid", aggfunc=len).reset_index().rename(columns={"sseqid": "hit_num"})
stat = pd.merge(conRes, stat[["qseqid", "qlen", "hit_num"]], left_on="id", right_on="qseqid", how="left")
stat["length"] = [float(i) for i in stat["length"]]
stat.loc[stat["hit_num"].isna(), "hit_num"] = 0
stat["hit_log10"] = [math.log10(i + 1) for i in stat["hit_num"]]
stat.to_csv(all_dir + "/blastCon_stat.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
#print(stat.shape)
#stat.head()
# plot
gp = ggplot(stat) + geom_vline(xintercept=17.5, color="grey", linetype="dashed") + geom_point(aes(x="length", y="hit_log10", color="asm"), alpha=0.6) + \
     facet_wrap("~asm", ncol=2) + scale_color_discrete(guide=False) + scale_x_continuous() + scale_y_continuous(limits=[0, stat["hit_log10"].max()]) +\
     xlab("Consensus repeat length") + ylab("Blast hit number (log10 + 1)")
gp.save((all_dir + "/blastCon_stat.pdf"), width=6, height=6, verbose=False)
# whether con repeats have hits
con_wht_num = len(stat[stat["hit_num"] > 0]["id"].unique())
con_tot_num = stat.shape[0]
print("{:.2%} ({} / {}) of consensus repeats have hits.".format(round(con_wht_num / con_tot_num, 3), con_wht_num, con_tot_num))
if con_wht_num != con_tot_num:
    print("Missed con repeats:")
    print(stat[stat["hit_num"] == 0].pivot_table(index="length", values="id", aggfunc=len).reset_index().rename(columns={"id": "con_num"}))
    stat[stat["hit_num"] == 0].head()

100.00% (237 / 237) of consensus repeats have hits.


In [15]:
# Blast res filtering
print(">", "Blast res filtering")
#stat_qt95_l = pd.read_csv(all_dir + "/blastConReo_stat_qt95.txt", header=0, sep="\t")
blcp_ct = pd.merge(blcp, stat_qt95_l, on="qlen", suffixes=["", "_qt95"])
# blcp_ct = pd.merge(blcp_ct, conRes[["id", "cid", "seq"]], left_on="qseqid", right_on="id").drop(columns="id")
# blcp_ct = pd.merge(blcp_ct, criPos, left_on="cid", right_on="cid")
# cads = []
# for _, row in blcp_ct.iterrows():
#     if row["contig"] == row["sseqid"]:
#         ant_l, ant_r = row["sstart"] - 1, row["send"]
#         if row["end"] <= ant_l:
#             cad = ant_l - row["end"]
#         elif row["start"] >= ant_r:
#             cad = - (row["start"] - ant_r)
#         else:
#             cad = 0
#     else:
#         cad = int(1e9)
#     cads.append(cad)
# blcp_ct["cadist"] = cads
# cutoff: pident and qcov
print(">> Filtering (pass the cutoff of pident and qcov)...")
blcp_ct_ftd = blcp_ct.loc[(blcp_ct["pident"] >= blcp_ct["pident_pred"]) & (blcp_ct["qcov"] >= blcp_ct["qcov_pred"])]
print("{} / {} hits passed.".format(blcp_ct_ftd.shape[0], blcp_ct.shape[0]))
# cutoff: in the same contig
# print(">> Filtering (in the same contig)...")
# blcp_ct_ftd = blcp_ct_ftd[blcp_ct_ftd["contig"] == blcp_ct_ftd["sseqid"]]
# print("{} / {} hits passed the filitering.".format(blcp_ct_ftd.shape[0], blcp_ct.shape[0]))
# cutoff: away from CRISPR arrays (corresponding CRISPR array)
# print(">> Filtering (away from CRISPR arrays)...")
# blcp_ct_ftd = blcp_ct_ftd[blcp_ct_ftd["cadist"].abs() > 0]
# print("{} / {} hits passed.".format(blcp_ct_ftd.shape[0], blcp_ct.shape[0]))
# write
blcp_ct_ftd.to_csv(all_dir + "/blastCon_ftd.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
blcp_ct_ftd.head()

> Blast res filtering
>> Filtering (pass the cutoff of pident and qcov)...
15029 / 43940 hits passed.


,asm,qseqid,sseqid,pident,length,mismatch,gap,qstart,qend,sstart,...,qcov,strand,qseq,sseq,alm,tool,pident_qt95,qcov_qt95,pident_pred,qcov_pred
0,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,100.0,30,0,0,1,30,79689,...,1.000,+,ATAAATGGTTCAGGGGATATTAATGGTGAT,ATAAATGGTTCAGGGGATATTAATGGTGAT,||||||||||||||||||||||||||||||,minCED,100.0,0.449455,99.973229,0.447913
1,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,100.0,27,0,0,1,27,79740,...,0.900,+,ATAAATGGTTCAGGGGATATTAATGGT,ATAAATGGTTCAGGGGATATTAATGGT,|||||||||||||||||||||||||||,minCED,100.0,0.449455,99.973229,0.447913
3,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000005.1,100.0,20,0,0,1,20,79638,...,0.667,+,ATAAATGGTTCAGGGGATAT,ATAAATGGTTCAGGGGATAT,||||||||||||||||||||,minCED,100.0,0.449455,99.973229,0.447913
6,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000045.1,100.0,15,0,0,10,24,15665,...,0.500,-,TCAGGGGATATTAAT,TCAGGGGATATTAAT,|||||||||||||||,minCED,100.0,0.449455,99.973229,0.447913
7,GCA_001594005.1,GCA_001594005.1_m_1_c1,LTBA01000021.1,100.0,14,0,0,16,29,28999,...,0.467,+,GATATTAATGGTGA,GATATTAATGGTGA,||||||||||||||,minCED,100.0,0.449455,99.973229,0.447913


In [16]:
# stat (query length ~ hit number)
stat_ftd = blcp_ct_ftd.pivot_table(index=["asm", "qseqid", "qlen"], values="sseqid", aggfunc=len).reset_index().rename(columns={"sseqid": "hit_num"})
stat_ftd = pd.merge(conRes, stat_ftd[["qseqid", "qlen", "hit_num"]], left_on="id", right_on="qseqid", how="left")
stat_ftd["length"] = [float(i) for i in stat_ftd["length"]]
stat_ftd.loc[stat_ftd["hit_num"].isna(), "hit_num"] = 0
stat_ftd["hit_log10"] = [math.log10(i + 1) for i in stat_ftd["hit_num"]]
stat_ftd.to_csv(all_dir + "/blastCon_stat_ftd.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
print(stat_ftd.shape)
#stat_ftd.head()
# plot
gp = ggplot(stat_ftd) + geom_vline(xintercept=17.5, color="grey", linetype="dashed") + geom_point(aes(x="length", y="hit_log10", color="asm"), alpha=0.6) + \
     facet_wrap("~asm", ncol=2) + scale_color_discrete(guide=False) + scale_x_continuous() + scale_y_continuous(limits=[0, stat_ftd["hit_log10"].max()]) +\
     xlab("Consensus repeat length") + ylab("Blast hit number (log10 + 1)")
gp.save((all_dir + "/blastCon_stat_ftd.pdf"), width=6, height=6, verbose=False)
# whether con repeats have hits
con_wht_num = len(stat_ftd[stat_ftd["hit_num"] > 0]["id"].unique())
con_tot_num = stat_ftd.shape[0]
print("{:.2%} ({} / {}) of con repeats have hits.".format(round(con_wht_num / con_tot_num, 3), con_wht_num, con_tot_num))
if con_wht_num != con_tot_num:
    print("Missed con repeats:")
    print(stat_ftd[stat_ftd["hit_num"] == 0].pivot_table(index="length", values="id", aggfunc=len).reset_index().rename(columns={"id": "con_num"}))
    display(stat_ftd[stat_ftd["hit_num"] == 0].head())

(237, 11)
99.20% (235 / 237) of con repeats have hits.
Missed con repeats:
   length  con_num
0    51.0        1
1    53.0        1


,contig,asm,tool,id,cid,seq,length,qseqid,qlen,hit_num,hit_log10
31,RGDI01000039.1,GCA_009918665.1,pilerCR,GCA_009918665.1_p_3_c1,GCA_009918665.1_p_3,TATGACTATGGCTTGAACTAGCATAACTGCCAGATGGTTGTTTACC...,53.0,NaN,NaN,0.0,0.0
48,RGDD01000047.1,GCA_009918765.1,pilerCR,GCA_009918765.1_p_2_c1,GCA_009918765.1_p_2,TCTAGATGGTAAACAACCATCTGGCAGTTATGCTAATTCAAGCCAT...,51.0,NaN,NaN,0.0,0.0


In [17]:
# Blast res -> Anti-repeat
print(">", "Blast res -> Anti-repeat")
antRes = blcp_ct_ftd[["sseqid", "sstart", "send", "strand", "asm", "tool", "qseqid", "sseq"]].reset_index(drop=True)
antRes["sstart"] = antRes["sstart"] - 1
antRes.insert(3, "id", "NA")
antRes.insert(4, "score", 900)
antRes.insert(9, "component", "antiRep")
antRes.insert(11, "length", antRes["sseq"].str.len())
antRes = antRes.rename(columns={"sseqid": "contig", "sstart": "start", "send": "end", "qseqid": "conid", "sseq": "seq"})
dfs = []
for conid, df in antRes.groupby("conid"):
    df["id"] = [conid + "_a" + str(i) for i in range(1, df.shape[0] + 1)]
    dfs.append(df)
antRes = pd.concat(dfs, ignore_index=True)
antRes.to_csv(all_dir + "/antiRep.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
print(antRes.shape)
antRes.head()

> Blast res -> Anti-repeat
(15029, 12)


,contig,start,end,id,score,strand,asm,tool,conid,component,seq,length
0,LTBA01000005.1,79688,79718,GCA_001594005.1_m_1_c1_a1,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRep,ATAAATGGTTCAGGGGATATTAATGGTGAT,30
1,LTBA01000005.1,79739,79766,GCA_001594005.1_m_1_c1_a2,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRep,ATAAATGGTTCAGGGGATATTAATGGT,27
2,LTBA01000005.1,79637,79657,GCA_001594005.1_m_1_c1_a3,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRep,ATAAATGGTTCAGGGGATAT,20
3,LTBA01000045.1,15664,15679,GCA_001594005.1_m_1_c1_a4,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRep,TCAGGGGATATTAAT,15
4,LTBA01000021.1,28998,29012,GCA_001594005.1_m_1_c1_a5,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRep,GATATTAATGGTGA,14


In [18]:
# CRISPR-close regions calc
print(">", "CRISPR-close regions calculation...")
slop_len = 100
print("Expand CRISPRs regions by +/- {} bp.".format(slop_len))
criSlp = slopReg(criMet, "lr", slop_len, genomeDir=asm_o_dir)
criSlp.to_csv(all_dir + "/criSlp.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)

> CRISPR-close regions calculation...
Expand CRISPRs regions by +/- 100 bp.


In [19]:
# Anti-repeat filitering
print(">", "Anti-repeat filitering...")
os.system("/bin/bash -c \"bedtools subtract -A -a <(tail -n +2 {}) -b <(tail -n +2 {}) > {}\"".format(all_dir + "/antiRep.txt", all_dir + "/criSlp.txt", all_dir + "/antiRep_ftd.txt"))
antRes_ftd = pd.read_csv(all_dir + "/antiRep_ftd.txt", header=None, sep="\t", quoting=csv.QUOTE_NONE)
antRes_ftd.columns = antRes.columns
print(antRes_ftd.shape)
#antRes_ftd.head()

> Anti-repeat filitering...
(11095, 12)


In [20]:
# Anti-repeat duplicates removing
print(">", "Anti-repeat duplicates removing...")
#antRes_ftd["occ_num"] = antRes_ftd.groupby(["asm", "contig", "start", "end"])["id"].transform("size")
#antRes_ftd[antRes_ftd["occ_num"] > 1].shape
antRes_ftd = antRes_ftd.drop_duplicates(["asm", "contig", "start", "end"])
print(antRes_ftd.shape)
#antRes_ftd.head()

> Anti-repeat duplicates removing...
(10046, 12)


In [33]:
# Anti-repeat flanking regions calc
print(">", "Anti-repeat flanking regions calculation")
atfRes = flankReg(antRes_ftd, mode="<.>", length=350, genomeDir=asm_o_dir, component="antiRepFlank")
#validateSeq(atfRes)
print(atfRes.shape)
display(atfRes.head())
# write
atfRes.to_csv(all_dir + "/antiRepFlank.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
recs = [SeqRecord.SeqRecord(Seq.Seq(row["seq"]), id=row["id"], description="") for i, row in atfRes.iterrows()]
_ = SeqIO.write(recs, all_dir + "/antiRepFlank.fa", "fasta")

> Anti-repeat flanking regions calculation
(20092, 14)


,contig,start,end,id,score,strand,asm,tool,conid,component,seq,length,length_ctg,oid
0,LTBA01000045.1,15314,15664,GCA_001594005.1_m_1_c1_a4_l,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRepFlank,ATAACTCTTCCACATACTTCAGAATTTTATTTAGATGCTGAAGTTA...,350,17162,GCA_001594005.1_m_1_c1_a4
0,LTBA01000045.1,15679,16029,GCA_001594005.1_m_1_c1_a4_r,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRepFlank,TGTACTTTTAATATTTATATCATTATTGAACGTTTCGTAATTCAAT...,350,17162,GCA_001594005.1_m_1_c1_a4
1,LTBA01000021.1,28648,28998,GCA_001594005.1_m_1_c1_a5_l,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRepFlank,ACACACTTCATAAGGAGTATGTGAATAATCCCATATCTCTTGAAAC...,350,45775,GCA_001594005.1_m_1_c1_a5
1,LTBA01000021.1,29012,29362,GCA_001594005.1_m_1_c1_a5_r,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,antiRepFlank,GTGGGCGTTATTAAAAAGATAAAAAAAGAGAAAAAATATAAGGCAT...,350,45775,GCA_001594005.1_m_1_c1_a5
2,LTBA01000010.1,48216,48566,GCA_001594005.1_m_2_c1_a3_l,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_2_c1,antiRepFlank,TCTTTTAGAAATATTAATATTCATAACCTTACTCAATTTATCTAAT...,350,79404,GCA_001594005.1_m_2_c1_a3


In [22]:
# Terminator identification
print(">", "Terminator identification")
tmn_out = all_dir + "/terminator_erpin.out"
tmn_err = all_dir + "/terminator_erpin.err"
if not os.path.exists(all_dir + "/terminator_erpin.out"):
    ret = os.system("scripts/Arnold/erpin scripts/Arnold/rho-indep.epn {} -1,4 -add 2 4 2 -pcw 3.0 -cutoff 100% > {} 2>{}".format(all_dir + "/antiRepFlank.fa", tmn_out, tmn_err))
    if ret != 0:
        print("Please confirm that erpin exists.", file=sys.stderr)
else:
    print("{} exists, skipped.".format(tmn_out))

> Terminator identification


In [23]:
# Terminator res extraction
print(">", "Terminator res extraction")
temRes = readERP("output/all/terminator_erpin.out")
print(">> Before filtering: {} / {} anti-repeat flanks contain terminators.".format(len(temRes["atf_id"].unique()), len(atfRes["id"].unique())))
print(temRes.shape)
temRes = temRes[temRes["strand"] == "+"]
print(">> After filtering: {} / {} anti-repeat flanks contain terminators.".format(len(temRes["atf_id"].unique()), len(atfRes["id"].unique())))
print(temRes.shape)
temRes.head()

> Terminator res extraction
>> Before filtering: 8539 / 20092 anti-repeat flanks contain terminators.
(13052, 8)
>> After filtering: 5247 / 20092 anti-repeat flanks contain terminators.
(6543, 8)


,id,atf_id,start,end,strand,seq,score,evalue
0,GCA_001594005.1_m_1_c1_a4_l_f1,GCA_001594005.1_m_1_c1_a4_l,216,246,+,GCCAGTAA.gt----------------------------.ATGCTG...,9.45,1.37e+03
1,GCA_001594005.1_m_1_c1_a4_l_f2,GCA_001594005.1_m_1_c1_a4_l,249,302,+,AAAACATT.tgcttgaattatttcccaggatat------.ATTGTT...,9.90,1.05e+03
3,GCA_001594005.1_m_1_c1_a4_r_f1,GCA_001594005.1_m_1_c1_a4_r,31,78,+,CGTTTCGT.aattcaatactattttac------------.CTGAAA...,9.96,1.02e+03
5,GCA_001594005.1_m_1_c1_a5_l_f1,GCA_001594005.1_m_1_c1_a5_l,12,64,+,AAGGAGTA.tgtgaataatcccatatctcttga------.AACTCT...,8.40,2.43e+03
7,GCA_001594005.1_m_1_c1_a5_r_f1,GCA_001594005.1_m_1_c1_a5_r,63,117,+,AAGTCATG.tgatttgggcaaaaagcttgaattta----.TATGGC...,8.30,2.56e+03


In [24]:
# stat
stat_tem = pd.DataFrame(temRes["atf_id"].value_counts())
round(stat_tem["atf_id"].mean(), 3)

1.247

In [32]:
# Terminator global position calc
print(">", "Terminator global position calculation")
tegRes = calcGlo(atfRes, temRes, genomeDir=asm_o_dir)
tegRes = tegRes[[i for i in tegRes.columns if not i.endswith("_tem")]]
tegRes.to_csv(all_dir + "/terminator.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
print(tegRes.shape)
tegRes.head()

> Terminator global position calculation
(6543, 16)


,contig,start,end,id,score,strand,asm,tool,conid,component,seq,length,length_ctg,oid,atf_id,evalue
0,LTBA01000045.1,15418,15449,GCA_001594005.1_m_1_c1_a4_l_f1,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,teminator,GCCAGTAAGTATGCTGGCTTATTTTGCTAAA,31,17162,GCA_001594005.1_m_1_c1_a4,GCA_001594005.1_m_1_c1_a4_l,1.37e+03
1,LTBA01000045.1,15362,15416,GCA_001594005.1_m_1_c1_a4_l_f2,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,teminator,AAAACATTTGCTTGAATTATTTCCCAGGATATATTGTTTTTTTCTT...,54,17162,GCA_001594005.1_m_1_c1_a4,GCA_001594005.1_m_1_c1_a4_l,1.05e+03
2,LTBA01000045.1,15709,15757,GCA_001594005.1_m_1_c1_a4_r_f1,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,teminator,CGTTTCGTAATTCAATACTATTTTACCTGAAACGCTTTTACCTTTTAT,48,17162,GCA_001594005.1_m_1_c1_a4,GCA_001594005.1_m_1_c1_a4_r,1.02e+03
3,LTBA01000021.1,28934,28987,GCA_001594005.1_m_1_c1_a5_l_f1,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,teminator,AAGGAGTATGTGAATAATCCCATATCTCTTGAAACTCTTTTTGTTG...,53,45775,GCA_001594005.1_m_1_c1_a5,GCA_001594005.1_m_1_c1_a5_l,2.43e+03
4,LTBA01000021.1,29074,29129,GCA_001594005.1_m_1_c1_a5_r_f1,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,teminator,AAGTCATGTGATTTGGGCAAAAAGCTTGAATTTATATGGCTTTTGT...,55,45775,GCA_001594005.1_m_1_c1_a5,GCA_001594005.1_m_1_c1_a5_r,2.56e+03


In [34]:
# TracrRNA calculation
print(">", "TracrRNA calculation")
traRes = unionReg(antRes_ftd, tegRes, genomeDir=asm_o_dir, component="tracrRNA")
traRes = traRes[[i for i in traRes.columns if not i.endswith("_teg")]].drop(columns=["oid", "atf_id", "length_ctg"])
traRes = pd.merge(traRes, conRes[["id", "cid"]], left_on="conid", right_on="id", suffixes=["", "_con"]).drop(columns="id_con")
traRes = pd.merge(traRes, criPos, on="cid", suffixes=["", "_cri"])
# distance between tracrRNA and corresponding CRISPR array
cads = []
for _, row in traRes.iterrows():
    if row["contig"] == row["contig_cri"]:
        if row["start"] >= row["end_cri"]:
            cad = row["start"] - row["end_cri"]
        elif row["end"] <= row["start_cri"]:
            cad = - (row["start_cri"] - row["end"])
        else:
            cad = 0
    else:
        cad = int(1e9)
    cads.append(cad)
traRes["ctdist"] = cads
traRes.to_csv(all_dir + "/tracrRNA.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
print(traRes.shape)
traRes.head()

> TracrRNA calculation
(6543, 20)


,contig,start,end,id,score,strand,asm,tool,conid,component,seq,length,evalue,cid,asm_cri,contig_cri,start_cri,end_cri,length_cri,ctdist
0,LTBA01000045.1,15418,15679,GCA_001594005.1_m_1_c1_a4_l_f1_t1,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,tracrRNA,TCAGGGGATATTAATATAACTCTTCCACATACTTCAGAATTTTATT...,261,1.37e+03,GCA_001594005.1_m_1,GCA_001594005.1,LTBA01000005.1,79637,79871,234,1000000000
1,LTBA01000045.1,15362,15679,GCA_001594005.1_m_1_c1_a4_l_f2_t1,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,tracrRNA,TCAGGGGATATTAATATAACTCTTCCACATACTTCAGAATTTTATT...,317,1.05e+03,GCA_001594005.1_m_1,GCA_001594005.1,LTBA01000005.1,79637,79871,234,1000000000
2,LTBA01000045.1,15664,15757,GCA_001594005.1_m_1_c1_a4_r_f1_t1,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,tracrRNA,ATTAATATCCCCTGATGTACTTTTAATATTTATATCATTATTGAAC...,93,1.02e+03,GCA_001594005.1_m_1,GCA_001594005.1,LTBA01000005.1,79637,79871,234,1000000000
3,LTBA01000021.1,28934,29012,GCA_001594005.1_m_1_c1_a5_l_f1_t1,900,-,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,tracrRNA,TCACCATTAATATCACACACTTCATAAGGAGTATGTGAATAATCCC...,78,2.43e+03,GCA_001594005.1_m_1,GCA_001594005.1,LTBA01000005.1,79637,79871,234,1000000000
4,LTBA01000021.1,28998,29129,GCA_001594005.1_m_1_c1_a5_r_f1_t1,900,+,GCA_001594005.1,minCED,GCA_001594005.1_m_1_c1,tracrRNA,GATATTAATGGTGAGTGGGCGTTATTAAAAAGATAAAAAAAGAGAA...,131,2.56e+03,GCA_001594005.1_m_1,GCA_001594005.1,LTBA01000005.1,79637,79871,234,1000000000


In [27]:
# TracrRNA Filtering 1
print(">", "TracrRNA Filtering 1")
print(">>", "Filtering (not contain N)")
traRes = traRes[~ traRes["seq"].str.contains("N")]
print(traRes.shape)
print(">>", "Filtering (not contain TTTT)")
traRes = traRes[~ traRes["seq"].str.contains("TTTT")]
print(traRes.shape)

> TracrRNA Filtering 1
>> Filtering (not contain N)
(6521, 20)
>> Filtering (not contain TTTT)
(1188, 20)


In [28]:
# TracrRNA Filtering 2
print(">", "TracrRNA Filtering 2")
os.system("/bin/bash -c \"bedtools subtract -A -a <(tail -n +2 {}) -b <(tail -n +2 {}) > {}\"".format(all_dir + "/tracrRNA.txt", all_dir + "/criSlp.txt", all_dir + "/tracrRNA_ftd.txt"))
traRes_ftd = pd.read_csv(all_dir + "/tracrRNA_ftd.txt", header=None, sep="\t", quoting=csv.QUOTE_NONE)
traRes_ftd.columns = traRes.columns
traRes_ftd.to_csv(all_dir + "/tracrRNA_ftd.txt", header=True, sep="\t", quoting=csv.QUOTE_NONE, index=False)
recs = [SeqRecord.SeqRecord(Seq.Seq(row["seq"]), id=row["id"], description="") for i, row in traRes_ftd.iterrows()]
SeqIO.write(recs, all_dir + "/tracrRNA_ftd.fa", "fasta")
print(traRes_ftd.shape)
traRes_ftd.head()

> TracrRNA Filtering 2
(1179, 20)


,contig,start,end,id,score,strand,asm,tool,conid,component,seq,length,evalue,cid,asm_cri,contig_cri,start_cri,end_cri,length_cri,ctdist
0,LTBA01000008.1,88113,88272,GCA_001594005.1_p_1_c1_a4_r_f1_t1,900,+,GCA_001594005.1,pilerCR,GCA_001594005.1_p_1_c1,tracrRNA,TTTCCATTAATATCTATATAACCTTGAGTATTACCATTGATTCTAA...,159,475.0,GCA_001594005.1_p_1,GCA_001594005.1,LTBA01000005.1,79736,79870,134,1000000000
1,RGDI01000306.1,1492,1563,GCA_009918665.1_m_14_c1_a8_l_f1_t1,900,-,GCA_009918665.1,minCED,GCA_009918665.1_m_14_c1,tracrRNA,CTGGGACTTCTGTTTCTCTCTCAGAGAAGATAACCCCGTATTTACT...,71,6550.0,GCA_009918665.1_m_14,GCA_009918665.1,RGDI01000363.1,229,377,148,1000000000
2,RGDI01000306.1,1551,1702,GCA_009918665.1_m_14_c1_a8_r_f1_t1,900,+,GCA_009918665.1,minCED,GCA_009918665.1_m_14_c1,tracrRNA,CAGAAGTCCCAGGTACCCGTAATACCTCTGTTGTCATTCATTCCTT...,151,1760.0,GCA_009918665.1_m_14,GCA_009918665.1,RGDI01000363.1,229,377,148,1000000000
3,RGDI01000111.1,4905,5138,GCA_009918665.1_p_1_c1_a2_r_f1_t1,900,+,GCA_009918665.1,pilerCR,GCA_009918665.1_p_1_c1,tracrRNA,GCTTGATATCCAACTGATACCGCATATTGTCCTTGTGTCGATGCTC...,233,9020.0,GCA_009918665.1_p_1,GCA_009918665.1,RGDI01000111.1,1964,2092,128,2813
4,RGDD01000801.1,6,126,GCA_009918765.1_m_13_c1_a2_l_f1_t1,900,-,GCA_009918765.1,minCED,GCA_009918765.1_m_13_c1,tracrRNA,CCTTGTGGACCTTGAGGTCCAGAAACACCTTGCGGTCCACTTGGTC...,120,4330.0,GCA_009918765.1_m_13,GCA_009918765.1,RGDD01000801.1,585,749,164,-459


In [29]:
# stat
traRes_ftd.pivot_table(index="asm", columns="tool", values="id", aggfunc=lambda x: len(x)).reset_index()

tool,asm,minCED,pilerCR
0,GCA_001594005.1,NaN,1.0
1,GCA_009918665.1,2.0,1.0
2,GCA_009918765.1,9.0,1.0
3,GCA_010672215.1,276.0,95.0
4,GCA_903819865.1,183.0,NaN
5,GCA_903830605.1,169.0,NaN
6,GCA_903838155.1,79.0,NaN
7,GCA_903853905.1,105.0,38.0
8,GCA_903901705.1,108.0,NaN
9,GCA_903907625.1,112.0,NaN


In [30]:
traRes_ftd[traRes_ftd["ctdist"].abs() < 1e6]

,contig,start,end,id,score,strand,asm,tool,conid,component,seq,length,evalue,cid,asm_cri,contig_cri,start_cri,end_cri,length_cri,ctdist
3,RGDI01000111.1,4905,5138,GCA_009918665.1_p_1_c1_a2_r_f1_t1,900,+,GCA_009918665.1,pilerCR,GCA_009918665.1_p_1_c1,tracrRNA,GCTTGATATCCAACTGATACCGCATATTGTCCTTGTGTCGATGCTC...,233,9020.0,GCA_009918665.1_p_1,GCA_009918665.1,RGDI01000111.1,1964,2092,128,2813
4,RGDD01000801.1,6,126,GCA_009918765.1_m_13_c1_a2_l_f1_t1,900,-,GCA_009918765.1,minCED,GCA_009918765.1_m_13_c1,tracrRNA,CCTTGTGGACCTTGAGGTCCAGAAACACCTTGCGGTCCACTTGGTC...,120,4330.0,GCA_009918765.1_m_13,GCA_009918765.1,RGDD01000801.1,585,749,164,-459
5,RGDD01000801.1,6,135,GCA_009918765.1_m_13_c1_a5_l_f1_t1,900,-,GCA_009918765.1,minCED,GCA_009918765.1_m_13_c1,tracrRNA,CCTTGTGGACCTTGTGGACCTTGAGGTCCAGAAACACCTTGCGGTC...,129,4330.0,GCA_009918765.1_m_13,GCA_009918765.1,RGDD01000801.1,585,749,164,-450
6,RGDD01000801.1,6,159,GCA_009918765.1_m_13_c1_a6_l_f1_t1,900,-,GCA_009918765.1,minCED,GCA_009918765.1_m_13_c1,tracrRNA,TGTGGACCTTGAGGACCAGCAATACCTTGTGGACCTTGTGGACCTT...,153,4330.0,GCA_009918765.1_m_13,GCA_009918765.1,RGDD01000801.1,585,749,164,-426
13,RGDD01000334.1,1850,2128,GCA_009918765.1_p_1_c1_a2_r_f1_t1,900,+,GCA_009918765.1,pilerCR,GCA_009918765.1_p_1_c1,tracrRNA,GCAGTTGACCCTTGTCTGGGCTGCTGCAAGCTCAGTACAAGTGTCA...,278,9210.0,GCA_009918765.1_p_1,GCA_009918765.1,RGDD01000334.1,805,986,181,864


In [31]:
# Stat for the whole process
print(">", "Stat for the whole process")
print(len(asms))
print(len(criRes["asm"].unique()))
print(len(antRes["asm"].unique()))
print(len(antRes_ftd["asm"].unique()))
print(len(tegRes["asm"].unique()))
print(len(traRes["asm"].unique()))
print(len(traRes_ftd["asm"].unique()))

> Stat for the whole process
14
12
12
10
10
10
10
